Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [ ]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=fc3ca188f96dbdc89c5eed51d606b2f3ad5417ab9a4663e1c027aa693cc038c7
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F

from vecstack import stacking

from sklearn import metrics




In [ ]:
'''
##################### CLASSES AND FUNCTIONS ##################################
'''
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

def standardize_data(df):
    scaler = RobustScaler()
    data = scaler.fit_transform(df)
    return data
#https://joedorfman.github.io/2018/numerai/

def my_loss_fn_1(y_true, y_pred):
    loss = tf.keras.losses.poisson(y_true, y_pred)
    return loss


def my_loss_fn_2(y_true, y_pred):
    threshold = 1     
    error = y_true - y_pred     
    is_small_error = tf.abs(error) <= threshold     
    small_error_loss = tf.square(error) / 2     
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

def my_loss_fn_3(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def rank_correlation(y_true, y_pred):
    return np.nan_to_num(stats.spearmanr(y_true, y_pred, axis=1)[0], nan=-1)


def rmsle_1(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

def rae_1(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


#https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/sklearn_example.py
#https://github.com/chricke/Numerai-Keras/tree/master
#https://forum.numer.ai/t/example-for-using-tensorflow-keras-with-a-custom-callback/855


'''
y_pred=[…]
y_true=[…]
corr_coefs=[…] #310 correlation coefficients with y_pred

mse= mean(square(y_pred - y_true)
correlation_penalty= square(std(abs(corr_coefs)))
loss= mse + correlation_penalty
'''

'\ny_pred=[…]\ny_true=[…]\ncorr_coefs=[…] #310 correlation coefficients with y_pred\n\nmse= mean(square(y_pred - y_true)\ncorrelation_penalty= square(std(abs(corr_coefs)))\nloss= mse + correlation_penalty\n'

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

./numerai_dataset_250.zip: 390MB [00:08, 46.0MB/s]                           
2021-02-07 16:44:19,307 INFO numerapi.base_api: unzipping file...


In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 6s, sys: 13.6 s, total: 1min 20s
Wall time: 1min 22s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
%%time
std_scale_train = StandardScaler().fit(training_data[feature_names])
df_std_train = std_scale_train.transform(training_data[feature_names])

std_scale_tournament = StandardScaler().fit(tournament_data[feature_names])
df_std_tournament = std_scale_tournament.transform(tournament_data[feature_names])

std_scale_validation = StandardScaler().fit(validation_data[feature_names])
df_std_validation = std_scale_validation.transform(validation_data[feature_names])

CPU times: user 14.6 s, sys: 2.93 s, total: 17.5 s
Wall time: 17.5 s


In [ ]:
%%time
df_std_train_f = pd.DataFrame(df_std_train, columns=feature_names)
df_std_tournament_f = pd.DataFrame(df_std_tournament, columns=feature_names)
df_std_validation_f = pd.DataFrame(df_std_validation, columns=feature_names)

CPU times: user 698 µs, sys: 1.22 ms, total: 1.92 ms
Wall time: 1.56 ms


In [ ]:
%%time
df_train = training_data
df_val = validation_data
df_tournament = tournament_data

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 10.5 µs


In [ ]:
%%time
df_train[feature_names] = df_std_train_f.values

CPU times: user 19.1 s, sys: 4.59 s, total: 23.6 s
Wall time: 23.6 s


In [ ]:
%%time
df_val[feature_names] = df_std_validation_f.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


CPU times: user 5.14 s, sys: 420 ms, total: 5.56 s
Wall time: 5.56 s


In [ ]:
%%time
df_tournament[feature_names] = df_std_tournament_f.values

CPU times: user 1min 1s, sys: 6.76 s, total: 1min 8s
Wall time: 1min 8s


In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=rand_2)

In [ ]:
##############################################################################
######################## MAKE PREDICTIONS ####################################
##############################################################################

In [ ]:

model_x = [
          
    LGBMRegressor(bagging_fraction = 0.94, feature_fraction = 0.10, max_depth = 24, min_child_weight = 24, min_split_gain = 0.08, num_leaves = 100),
    LGBMRegressor(learning_rate = 0.1, n_estimators = 40),
    XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, alpha=0.1),
    XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=50, colsample_bytree=0.5, alpha=0.1),
    XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=50, colsample_bytree=0.5)

]

from sklearn.metrics import explained_variance_score

S_train, S_test = stacking(model_x,                   
                          X_train, y_train, X_test,   
                          regression=True, 
                          mode='oof_pred_bag',
                          needs_proba=False,
                          save_dir=None,
                          metric= explained_variance_score,
                          n_folds=7,
                          stratified=True,
                          shuffle=True,
                          random_state=rand_3,verbose=2)


del model_x

gc.collect()

/usr/local/lib/python3.6/dist-packages/vecstack/core.py:461: UserWarning: This is regression task hence classification-specific parameters set to <True> were ignored: <stratified>
  warnings.warn(warn_str, UserWarning)


task:         [regression]
metric:       [explained_variance_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBRegressor]
[16:48:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
    fold  0:  [0.00287870]
[16:49:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
    fold  1:  [0.00361597]
[16:51:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
    fold  2:  [0.00326407]
[16:52:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
    fold  3:  [0.00325766]
[16:54:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
    fold  4:  [0.00341747]
    ----
    MEAN:     [0.00328677] + [0.00024221]
    FULL:     [0.00328521]

model 

127

In [ ]:
model_e = LGBMRegressor(random_state=rand_1)
gc.collect()

0

In [ ]:
model_1 = LGBMRegressor(bagging_fraction = 0.94, feature_fraction = 0.10, max_depth = 24, min_child_weight = 24, min_split_gain = 0.08, num_leaves = 100)
model_1.fit(training_data[feature_names], training_data[TARGET_NAME])

gc.collect()

[17:12:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


16

In [ ]:
model_2 = LGBMRegressor(learning_rate = 0.1, n_estimators = 40)
model_2.fit(training_data[feature_names], training_data[TARGET_NAME])

gc.collect()

[17:14:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


16

In [ ]:
model_3 = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.1, alpha=0.1)
model_3.fit(training_data[feature_names], training_data[TARGET_NAME])

gc.collect()

[17:17:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


16

In [ ]:
model_4 = XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=50, colsample_bytree=0.5, alpha=0.1)
model_4.fit(training_data[feature_names], training_data[TARGET_NAME])

gc.collect()

In [ ]:
model_5 = XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=50, colsample_bytree=0.5)
model_5.fit(training_data[feature_names], training_data[TARGET_NAME])

gc.collect()

In [ ]:
y_pred_model_1 = model_1.predict(tournament_data[feature_names])
gc.collect()

2

In [ ]:
y_pred_model_2 = model_2.predict(tournament_data[feature_names])
gc.collect()

2

In [ ]:
y_pred_model_3 = model_3.predict(tournament_data[feature_names])
gc.collect()

2

In [ ]:
y_pred_model_4 = model_4.predict(tournament_data[feature_names])
gc.collect()

In [ ]:
y_pred_model_5 = model_5.predict(tournament_data[feature_names])
gc.collect()

In [ ]:
df_mix = pd.DataFrame(columns=['f0','f1','f2','f3','f4'])

In [ ]:
df_mix['f0'] = y_pred_model_1.astype(np.float)
df_mix['f1'] = y_pred_model_2.astype(np.float)
df_mix['f2'] = y_pred_model_3.astype(np.float)
df_mix['f3'] = y_pred_model_3.astype(np.float)
df_mix['f4'] = y_pred_model_3.astype(np.float)

In [ ]:
model_e = model_e.fit(S_train, y_train)
y_pred = model_e.predict(S_test)
gc.collect()

33

In [ ]:
y_pred_final = model_e.predict(df_mix)
tournament_data[PREDICTION_NAME] = y_pred_final
gc.collect()

4

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.491529
n000920ed083903f    0.483494
n0038e640522c4a6    0.540378
n004ac94a87dc54b    0.500574
n0052fe97ea0c05f    0.495001
                      ...   
nffe93b9f7d7530d    0.460178
nffef5e57638d46e    0.502362
nfff640225894e11    0.479273
nfff6e21335c357f    0.511662
nfffc6ad6d850c65    0.495001
Name: prediction, Length: 1644415, dtype: float64

In [ ]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [ ]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.491529
n000920ed083903f    0.483494
n0038e640522c4a6    0.540378
n004ac94a87dc54b    0.500574
n0052fe97ea0c05f    0.495001
                      ...   
nffe93b9f7d7530d    0.460178
nffef5e57638d46e    0.502362
nfff640225894e11    0.479273
nfff6e21335c357f    0.511662
nfffc6ad6d850c65    0.495001
Name: prediction, Length: 1644415, dtype: float64

In [ ]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.10)

In [ ]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

2021-02-07 18:16:05,556 INFO numexpr.utils: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-02-07 18:16:05,557 INFO numexpr.utils: NumExpr defaulting to 8 threads.


In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.470088
n000920ed083903f    0.413908
n0038e640522c4a6    0.681014
n004ac94a87dc54b    0.537499
n0052fe97ea0c05f    0.477078
                      ...   
nffe93b9f7d7530d    0.440151
nffef5e57638d46e    0.496154
nfff640225894e11    0.479012
nfff6e21335c357f    0.690090
nfffc6ad6d850c65    0.528811
Name: prediction, Length: 1644415, dtype: float64

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [ ]:
# NameOfYourAI FISAGOL_2: sescervello9@gmail.com
public_id = "YYGB2JJQG2ZTB72WVUHJJASCFUBK656A"
secret_key = "L6FF4CD6ROSSEMWZVTXBBLRHICBASW3CJVELD5IFZR7QCI46XBJLEOHZU7HCZ2RB"
model_id = "c6f3339f-f677-48a0-ac42-2995afc614a5"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
print("Uploading DataFrame in Numerai...")
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)
print("DataFrame Uploaded...")

2021-02-07 18:16:09,220 INFO numerapi.base_api: uploading predictions...


Uploading DataFrame in Numerai...
DataFrame Uploaded...


In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol_2.csv')
!cp FisaGol_2.csv "drive/My Drive/"